<a href="https://colab.research.google.com/github/jc533/Song_data_game/blob/attackofthetitan-patch-1/datagame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import

In [46]:
import sys
!cp ../input/rapids/rapids.0.14.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.10/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.10"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/
import warnings
warnings.filterwarnings('ignore')

cp: cannot stat '../input/rapids/rapids.0.14.0': No such file or directory
tar (child): rapids.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
cp: cannot stat '/opt/conda/envs/rapids/lib/libxgboost.so': No such file or directory


In [47]:
import pandas as pd
import numpy as np
import os
# from google.colab import drive

!pip install missingno
import missingno as msno

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
# drive.mount('/content/drive')
!pip3 install lightgbm
import lightgbm as lgbm

# data viz
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
import plotly.express as px
%matplotlib inline

In [48]:
import warnings
warnings.filterwarnings("ignore")

# read file

In [49]:
path = '/kaggle/input/datagame/datagame-2023/'
train = pd.read_parquet(f'{path}/label_train_source.parquet')
test = pd.read_parquet(f'{path}/label_test_source.parquet')
target = pd.read_parquet(f'{path}/label_train_target.parquet')
composer = pd.read_parquet(f'{path}/meta_song_composer.parquet')
genre = pd.read_parquet(f'{path}/meta_song_genre.parquet')
lyricist = pd.read_parquet(f'{path}/meta_song_lyricist.parquet')
producer = pd.read_parquet(f'{path}/meta_song_producer.parquet')
titletext = pd.read_parquet(f'{path}/meta_song_titletext.parquet')
song = pd.read_parquet(f'{path}/meta_song.parquet')


# merge data


In [50]:
#song_data = pd.concat([train, target])
#song_data = song_data.sort_values(by=['session_id', 'listening_order'])
#song_data = pd.merge(train, target, on='session_id', how='inner')
song_data = pd.merge(train, song, on='song_id', how='left')
song_data = pd.merge(song_data, composer, on='song_id', how='left')
song_data = pd.merge(song_data, genre, on='song_id', how='left')
song_data = pd.merge(song_data, lyricist, on='song_id', how='left')
song_data = pd.merge(song_data, producer, on='song_id', how='left')
song_data = pd.merge(song_data, titletext, on='song_id', how='left')
song_vali = pd.merge(target, song, on='song_id', how='left')
song_vali = pd.merge(song_vali, composer, on='song_id', how='left')
song_vali = pd.merge(song_vali, genre, on='song_id', how='left')
song_vali = pd.merge(song_vali, lyricist, on='song_id', how='left')
song_vali = pd.merge(song_vali, producer, on='song_id', how='left')
song_vali = pd.merge(song_vali, titletext, on='song_id', how='left')


In [51]:
song_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52895871 entries, 0 to 52895870
Data columns (total 16 columns):
 #   Column           Dtype  
---  ------           -----  
 0   session_id       int64  
 1   song_id          object 
 2   unix_played_at   int64  
 3   play_status      int64  
 4   login_type       int64  
 5   listening_order  int64  
 6   artist_id        float64
 7   song_length      float64
 8   album_id         float64
 9   language_id      float64
 10  album_month      object 
 11  composer_id      object 
 12  genre_id         object 
 13  lyricist_id      object 
 14  producer_id      object 
 15  title_text_id    object 
dtypes: float64(4), int64(5), object(7)
memory usage: 6.3+ GB


In [52]:
song_vali.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13159149 entries, 0 to 13159148
Data columns (total 16 columns):
 #   Column           Dtype  
---  ------           -----  
 0   session_id       int64  
 1   song_id          object 
 2   unix_played_at   int64  
 3   play_status      int64  
 4   login_type       int64  
 5   listening_order  int64  
 6   artist_id        float64
 7   song_length      float64
 8   album_id         float64
 9   language_id      float64
 10  album_month      object 
 11  composer_id      object 
 12  genre_id         object 
 13  lyricist_id      object 
 14  producer_id      object 
 15  title_text_id    object 
dtypes: float64(4), int64(5), object(7)
memory usage: 1.6+ GB


In [53]:
song_data.drop_duplicates(subset=['session_id', 'song_id', 'listening_order'], inplace=True)
song_vali.drop_duplicates(subset=['session_id', 'song_id', 'listening_order'], inplace=True)

In [54]:
for i in song_data.select_dtypes(include=['object']).columns:
  song_data[i][song_data[i].isnull()] = 'unknown'
song_data = song_data.fillna(value=0)

for i in song_vali.select_dtypes(include=['object']).columns:
  song_vali[i][song_vali[i].isnull()] = 'unknown'
song_vali = song_vali.fillna(value=0)

label_encoder = LabelEncoder()
# one_hot = OneHotEncoder()

for i in song_data.columns:
  song_data[i] = label_encoder.fit_transform(song_data[i])
for i in song_vali.columns:
  song_vali[i] = label_encoder.fit_transform(song_vali[i])
song_data.reset_index(drop=True)
song_vali.reset_index(drop=True)

,session_id,song_id,unix_played_at,play_status,login_type,listening_order,artist_id,song_length,album_id,language_id,album_month,composer_id,genre_id,lyricist_id,producer_id,title_text_id
0,246,161432,226329,2,3,0,5178,232,57005,1,666,42322,25,27743,22667,754
1,246,17592,226331,2,3,1,5178,232,0,0,712,36282,38,23754,712,75
2,246,106096,226392,2,3,2,2225,278,20032,1,615,59876,30,39204,15615,754
3,246,201297,226438,1,3,3,5178,255,0,0,712,63764,38,41766,22667,75
4,246,25929,226441,2,3,4,5178,237,0,0,712,1921,38,27743,712,174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2861290,571663,117218,2177021,1,2,0,50531,116,123183,8,698,63764,17,41766,22667,798
2861291,571663,159104,2177026,1,2,1,50531,68,123183,8,698,63764,16,41766,22667,798
2861292,571663,83258,2177035,1,2,2,50531,63,123183,8,698,63764,16,41766,22667,798
2861293,571663,101811,2177036,1,2,3,50531,63,123183,8,698,63764,17,41766,22667,798


In [106]:
song_data.shape[0]/20
song_vali.shape[0]/5

572259.0

In [56]:
training = song_data.groupby('session_id')
targeting = song_vali.groupby('session_id')

#training = song_data.merge()

In [57]:
targeting.groups

{0: [1226645, 1226647, 1226648, 1226649, 1226657], 1: [4328443, 4328445, 4328453, 4328455, 4328459], 2: [3576942, 3576952, 3576954, 3576960, 3576961], 3: [9020097, 9020099, 9020100, 9020102, 9020103], 4: [9908407, 9908408, 9908410, 9908411, 9908413], 5: [11234695, 11234696, 11234697, 11234698, 11234706], 6: [4750700, 4750701, 4750705, 4750707, 4750708], 7: [8424220, 8424224, 8424228, 8424232, 8424233], 8: [5568381, 5568383, 5568385, 5568387, 5568393], 9: [1637081, 1637087, 1637089, 1637091, 1637093], 10: [8576390, 8576392, 8576394, 8576396, 8576397], 11: [11690954, 11690956, 11690958, 11690960, 11690962], 12: [11571501, 11571503, 11571505, 11571506, 11571508], 13: [11448581, 11448582, 11448584, 11448585, 11448586], 14: [2175454, 2175455, 2175456, 2175457, 2175458], 15: [1667207, 1667209, 1667211, 1667213, 1667215], 16: [2974401, 2974402, 2974403, 2974404, 2974405], 17: [5676115, 5676117, 5676119, 5676121, 5676123], 18: [9045615, 9045616, 9045618, 9045624, 9045626], 19: [9088171, 908817

In [58]:
targeting.get_group(751)

,session_id,song_id,unix_played_at,play_status,login_type,listening_order,artist_id,song_length,album_id,language_id,album_month,composer_id,genre_id,lyricist_id,producer_id,title_text_id
8659344,751,274093,568179,1,3,0,16443,260,0,0,712,22434,38,5819,22667,146
8659345,751,311304,568230,1,3,1,19089,301,74036,3,573,3441,10,41766,22667,948
8659347,751,311304,568239,1,3,2,19089,301,74036,3,573,3441,10,41766,22667,948
8659349,751,327888,568255,1,3,3,84,227,0,0,712,63764,38,41766,22667,1093
8659350,751,318261,568258,1,3,4,84,301,0,0,712,63764,38,41766,22667,1093


# Preprocess


In [59]:
for i in test.columns:
  test[i] = label_encoder.fit_transform(test[i])
test.head(50)


,session_id,song_id,unix_played_at,play_status,login_type,listening_order
0,118,89565,299096,2,2,0
1,118,194933,299203,2,2,1
2,118,109275,299237,2,2,2
3,118,98197,299238,2,2,3
4,118,250870,299240,2,2,4
5,118,154531,299241,2,2,5
6,118,24559,299242,2,2,6
7,118,183641,299243,2,2,7
8,118,49060,299245,2,2,8
9,118,243906,299246,2,2,9


In [60]:
training.get_group(1)

,session_id,song_id,unix_played_at,play_status,login_type,listening_order,artist_id,song_length,album_id,language_id,album_month,composer_id,genre_id,lyricist_id,producer_id,title_text_id
8886521,1,9099,1544057,2,2,0,0,0,73963,1,698,102271,24,65290,34243,2055
8886523,1,164152,1544197,2,2,1,0,0,111424,1,716,102271,29,65290,34243,2055
8886525,1,618208,1544229,2,2,2,0,0,188294,1,738,102271,29,65290,34243,2055
8886527,1,71629,1544245,2,2,3,79584,253,185280,1,737,36069,24,29136,14984,1393
8886531,1,574233,1544251,1,2,4,0,0,95719,1,709,102271,24,65290,34243,2055
8886533,1,219549,1544252,2,2,5,7660,288,0,0,760,26217,37,52784,23411,1393
8886535,1,372056,1544256,2,2,6,7155,258,0,0,760,102271,37,65290,34243,98
8886536,1,90818,1544258,2,2,7,42697,220,110407,1,716,83954,29,53756,14984,1393
8886538,1,428781,1544263,2,2,8,786,267,142388,1,726,72743,24,7440,14984,1393
8886542,1,499369,1544268,2,2,9,5679,245,92359,1,708,39412,24,25629,10292,1032


In [61]:
targeting.get_group(1)

,session_id,song_id,unix_played_at,play_status,login_type,listening_order,artist_id,song_length,album_id,language_id,album_month,composer_id,genre_id,lyricist_id,producer_id,title_text_id
4328443,1,282056,641313,2,2,0,14733,255,16187,1,607,13852,25,36300,15615,754
4328445,1,178483,641316,2,2,1,22939,234,70061,1,675,37013,25,13938,10018,1010
4328453,1,326023,641319,2,2,2,0,0,108884,1,695,63764,30,41766,22667,1481
4328455,1,35198,641321,2,2,3,20887,221,67276,1,674,37362,30,25280,10018,1010
4328459,1,122868,641323,2,2,4,32648,191,63193,2,671,60433,6,39588,10018,1010


In [100]:
%%time
t = {}
# t = {i: training.get_group(i).to_numpy() for i in training.groups}
# df_t = pd.DataFrame(t)
k=0
s=100000
# s=1
size = len(song_data)/20
itm = training.groups.items()
for sess_id, group_indices in itm:
    tmp = song_data.loc[group_indices].to_numpy()
    tmp = tmp.tolist()
#     print(type(song_data.loc[group_indices].to_nump))
    t[sess_id] = tmp
    if k%s==0 and k>=s:
#         break
        print(k,k/size*100)
    k+=1
df_t = pd.DataFrame(t)

100000 17.474605030239804
200000 34.94921006047961
300000 52.423815090719415
400000 69.89842012095922
500000 87.37302515119903
CPU times: user 9min 18s, sys: 15.2 s, total: 9min 33s
Wall time: 9min 35s


In [ ]:
%%time
v = {}
# t = {i: training.get_group(i).to_numpy() for i in training.groups}
# df_t = pd.DataFrame(t)
k=0
s=100000
# s=1
size = len(song_vali)/5
itm = targeting.groups.items()
for sess_id, group_indices in itm:
    tmp = song_vali.loc[group_indices].to_numpy()
    tmp = tmp.tolist()
#     print(type(song_data.loc[group_indices].to_nump))
    v[sess_id] = tmp
    if k%s==0 and k>=s:
#         break
        print(k,k/size*100)
    k+=1
df_v = pd.DataFrame(v)


# Train

In [102]:
x = df_t.transpose() #= pd.DataFrame.from_dict(t)
y = df_v.transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,"[0, 627751, 589474, 1, 2, 0, 23962, 224, 42255...","[0, 312752, 589651, 1, 2, 1, 0, 0, 0, 0, 760, ...","[0, 248305, 591600, 1, 2, 2, 45870, 175, 80275...","[0, 320472, 591754, 1, 2, 3, 6603, 265, 40053,...","[0, 247559, 592219, 1, 2, 4, 8094, 275, 156571...","[0, 435685, 592220, 1, 2, 5, 47254, 242, 0, 0,...","[0, 231908, 592224, 1, 2, 6, 1351, 254, 0, 0, ...","[0, 101178, 592224, 1, 2, 7, 61160, 232, 0, 0,...","[0, 537561, 592226, 1, 2, 8, 22262, 259, 12674...","[0, 73571, 592228, 1, 2, 9, 49480, 246, 106454...","[0, 380610, 592230, 1, 2, 10, 0, 0, 109594, 1,...","[0, 284722, 592231, 1, 2, 11, 2451, 207, 0, 0,...","[0, 168186, 592233, 1, 2, 12, 1650, 250, 12209...","[0, 595275, 592235, 1, 2, 13, 1351, 289, 0, 0,...","[0, 570366, 592237, 1, 2, 14, 17272, 260, 0, 0...","[0, 173137, 592238, 1, 2, 15, 9263, 280, 0, 0,...","[0, 358093, 592240, 1, 2, 16, 1533, 287, 0, 0,...","[0, 109434, 592241, 1, 2, 17, 11796, 236, 0, 0...","[0, 313856, 592436, 1, 2, 18, 5605, 208, 6234,...","[0, 330504, 592610, 1, 2, 19, 2026, 220, 0, 0,..."
1,"[1, 9099, 1544057, 2, 2, 0, 0, 0, 73963, 1, 69...","[1, 164152, 1544197, 2, 2, 1, 0, 0, 111424, 1,...","[1, 618208, 1544229, 2, 2, 2, 0, 0, 188294, 1,...","[1, 71629, 1544245, 2, 2, 3, 79584, 253, 18528...","[1, 574233, 1544251, 1, 2, 4, 0, 0, 95719, 1, ...","[1, 219549, 1544252, 2, 2, 5, 7660, 288, 0, 0,...","[1, 372056, 1544256, 2, 2, 6, 7155, 258, 0, 0,...","[1, 90818, 1544258, 2, 2, 7, 42697, 220, 11040...","[1, 428781, 1544263, 2, 2, 8, 786, 267, 142388...","[1, 499369, 1544268, 2, 2, 9, 5679, 245, 92359...","[1, 428781, 1544271, 2, 2, 10, 786, 267, 14238...","[1, 499369, 1544303, 2, 2, 11, 5679, 245, 9235...","[1, 585530, 1544305, 2, 2, 12, 938, 264, 11491...","[1, 549425, 1544308, 1, 2, 13, 22262, 266, 965...","[1, 394189, 1544399, 2, 2, 14, 36094, 213, 111...","[1, 174375, 1544402, 2, 2, 15, 54736, 278, 127...","[1, 301706, 1544405, 2, 2, 16, 22262, 322, 160...","[1, 443298, 1544409, 2, 2, 17, 48881, 243, 170...","[1, 123111, 1544411, 2, 2, 18, 9469, 207, 7971...","[1, 251506, 1544413, 2, 2, 19, 0, 0, 108586, 1..."
2,"[2, 276176, 3409065, 2, 2, 0, 0, 0, 190229, 12...","[2, 606077, 3409250, 2, 2, 1, 0, 0, 171436, 12...","[2, 114740, 3409448, 2, 2, 2, 57644, 140, 1750...","[2, 354290, 3409627, 2, 2, 3, 19099, 161, 1915...","[2, 425654, 3409766, 2, 2, 4, 52033, 178, 1639...","[2, 486801, 3410055, 2, 2, 5, 81210, 214, 1925...","[2, 493918, 3410230, 2, 2, 6, 4711, 217, 11141...","[2, 265461, 3410412, 2, 2, 7, 76357, 218, 1723...","[2, 602648, 3410578, 2, 2, 8, 25919, 203, 1762...","[2, 3997, 3420355, 2, 2, 9, 36502, 167, 181615...","[2, 130118, 3420487, 2, 2, 10, 83513, 229, 202...","[2, 181719, 3420661, 2, 2, 11, 37411, 146, 208...","[2, 313155, 3420763, 2, 2, 12, 0, 0, 180063, 1...","[2, 138864, 3421156, 2, 2, 13, 0, 0, 167865, 1...","[2, 68718, 3421577, 2, 2, 14, 40805, 191, 1739...","[2, 1743, 3421707, 2, 2, 15, 14305, 195, 16988...","[2, 264919, 3422815, 2, 2, 16, 78545, 170, 180...","[2, 583280, 3423227, 2, 2, 17, 65207, 195, 205...","[2, 592288, 3423659, 2, 2, 18, 37111, 199, 197...","[2, 150475, 3423820, 2, 2, 19, 0, 0, 33074, 8,..."
3,"[3, 28195, 683315, 2, 13, 0, 342, 233, 71, 4, ...","[3, 574519, 683315, 2, 13, 1, 158, 233, 2261, ...","[3, 387204, 683317, 2, 13, 2, 7, 250, 7, 1, 65...","[3, 547407, 683317, 2, 13, 3, 115, 240, 870, 4...","[3, 219286, 683529, 2, 13, 4, 0, 0, 0, 0, 760,...","[3, 611194, 683542, 2, 13, 5, 158, 265, 6156, ...","[3, 518061, 870160, 2, 13, 6, 26581, 262, 0, 0...","[3, 527548, 870386, 2, 13, 7, 7280, 256, 25711...","[3, 152014, 870398, 2, 13, 8, 63751, 222, 0, 0...","[3, 411956, 870601, 2, 13, 9, 4796, 207, 5031,...","[3, 603186, 870777, 2, 13, 10, 8003, 269, 1968...","[3, 259818, 870789, 2, 13, 11, 1315, 227, 1560...","[3, 186864, 870802, 2, 13, 12, 144, 229, 5398,...","[3, 259818, 870810, 2, 13, 13, 1315, 227, 1560...","[3, 186864, 870996, 2, 13, 14, 144, 229, 5398,...","[3, 186864, 871101, 2, 13, 15, 144, 229,

In [62]:
X = training.drop(columns=['song_id'])
y = training.song_id
X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size = 0.25, random_state = 0)

AttributeError: 'DataFrameGroupBy' object has no attribute 'drop'

In [ ]:
params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'gbdt',
        'learning_rate': 0.3 ,
        'verbose': 0,
        'num_leaves': 108,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.9
        'feature_fraction_seed': 1,
        'max_bin': 256,
        'max_depth': 10,
        'num_rounds': 200,
        'metric' : 'auc'
    }

%time model_f1 = lgbm.train(params, train_set=train_set,  valid_sets=train_set, verbose_eval=5)